In [ ]:
from keras.models import Sequential
from keras.layers import Conv2D, ZeroPadding2D, Activation, Input, concatenate
from keras.models import Model
from keras.layers.normalization import BatchNormalization
from keras.layers.pooling import MaxPooling2D, AveragePooling2D
from keras.layers.merge import Concatenate
from keras.layers.core import Lambda, Flatten, Dense
from keras.initializers import glorot_uniform
from keras.engine.topology import Layer
from keras import backend as K

#------------用于绘制模型细节，可选--------------#
from IPython.display import SVG
from keras.utils.vis_utils import model_to_dot
from keras.utils import plot_model
#------------------------------------------------#

K.set_image_data_format('channels_first')

import time
import cv2
import os
import numpy as np
from numpy import genfromtxt
import pandas as pd
import tensorflow as tf
import fr_utils
from inception_blocks_v2 import *

%matplotlib inline
%load_ext autoreload
%autoreload 2

import sys
np.set_printoptions(np.set_printoptions(threshold=sys.maxsize))

In [ ]:
#获取模型
FRmodel = faceRecoModel(input_shape=(3,96,96))

#打印模型的总参数数量
print("参数数量：" + str(FRmodel.count_params()))

In [ ]:
def triplet_loss(y_true, y_pred, alpha = 0.2):
    """
    三元组loss，y_true不用，y_pred是一个列表，分别是A、P、N，样本个数未定，每列是一个128维向量。
    人脸验证、识别：两个图片输入网络，分别计算出对应的128维向量，然后比较这两个向量的距离，用于判断是不是同一个人
    """
    anchor, positive, negative = y_pred[0], y_pred[1], y_pred[2]
    pos_dist = tf.reduce_sum(tf.square(tf.subtract(anchor, positive)), axis = -1)
    neg_dist = tf.reduce_sum(tf.square(tf.subtract(anchor, negative)), axis = -1)
    basic_loss = tf.add(tf.subtract(pos_dist, neg_dist), alpha)  
    loss = tf.reduce_sum(tf.maximum(basic_loss, 0))
    return loss

In [ ]:
with tf.Session() as test:
    tf.set_random_seed(1)
    y_true = (None, None, None)
    y_pred = (tf.random_normal([3, 128], mean=6, stddev=0.1, seed = 1),
              tf.random_normal([3, 128], mean=1, stddev=1, seed = 1),
              tf.random_normal([3, 128], mean=3, stddev=4, seed = 1))
    loss = triplet_loss(y_true, y_pred)

    print("loss = " + str(loss.eval()))

In [ ]:
"""
加载训练好的模型
"""
#开始时间
start_time = time.clock()

#编译模型
FRmodel.compile(optimizer = 'adam', loss = triplet_loss, metrics = ['accuracy'])

#加载权值  也可以训练出权重
fr_utils.load_weights_from_FaceNet(FRmodel)

#结束时间
end_time = time.clock()

#计算时差
minium = end_time - start_time

print("执行了：" + str(int(minium / 60)) + "分" + str(int(minium%60)) + "秒")

人脸验证

In [ ]:
# 处理数据库，把每个人脸映射到一个128维的向量
database = {}
database["danielle"] = fr_utils.img_to_encoding("images/danielle.png", FRmodel)
database["younes"] = fr_utils.img_to_encoding("images/younes.jpg", FRmodel)
database["tian"] = fr_utils.img_to_encoding("images/tian.jpg", FRmodel)
database["andrew"] = fr_utils.img_to_encoding("images/andrew.jpg", FRmodel)
database["kian"] = fr_utils.img_to_encoding("images/kian.jpg", FRmodel)
database["dan"] = fr_utils.img_to_encoding("images/dan.jpg", FRmodel)
database["sebastiano"] = fr_utils.img_to_encoding("images/sebastiano.jpg", FRmodel)
database["bertrand"] = fr_utils.img_to_encoding("images/bertrand.jpg", FRmodel)
database["kevin"] = fr_utils.img_to_encoding("images/kevin.jpg", FRmodel)
database["felix"] = fr_utils.img_to_encoding("images/felix.jpg", FRmodel)
database["benoit"] = fr_utils.img_to_encoding("images/benoit.jpg", FRmodel)
database["arnaud"] = fr_utils.img_to_encoding("images/arnaud.jpg", FRmodel)

In [ ]:
def verify(image_path, identity, database, model):
    encoding = fr_utils.img_to_encoding(image_path, FRmodel)
    dist = np.linalg.norm(encoding - database[identity])
    if dist < 0.7:
        print("welcome ," + str(identity))
        is_open_door = True
    else:
        print("unconform, sorry," + str(identity))
        is_open_door = False
    return dist, is_open_door

In [ ]:
verify("images/camera_0.jpg","younes",database,FRmodel)

人脸识别

In [ ]:
def who_is_it(image_path, database, model):
    encoding = fr_utils.img_to_encoding(image_path, FRmodel)
    min_dist = 100
    for name, database_enc in database.items():
        dis = np.linalg.norm(encoding - database_enc)
        
        if dis < min_dist:
            min_dist = dis
            identity = name
    if min_dist < 0.7:
        print("welecome," + str(identity))
    else:
        print("sorry, illegal")
    
    return min_dist, identity

In [ ]:
who_is_it("images/camera_0.jpg", database, FRmodel)